<a href="https://colab.research.google.com/github/KieferN/Batch_Normalization/blob/master/BatchNorm_Inception_Cifa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Inception Net with Batch Normalization

# Imports

In [0]:
%load_ext tensorboard

from keras.datasets import cifar10
from keras.utils import np_utils
from keras.layers import Input, Conv2D, MaxPool2D, concatenate, \
BatchNormalization, Dense, Activation, \
GlobalAveragePooling2D, Dropout
from keras.models import Model
from keras.utils import plot_model
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator

from datetime import datetime
from keras.callbacks import TensorBoard, LearningRateScheduler

import math


# Preparations

Load Input Data \\
(For more information on CIFAR10: https://www.cs.toronto.edu/~kriz/cifar.html)

In [0]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

Normalize Input Data

In [0]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

input_img = Input(shape = (32, 32, 3))

Create Filesystem Structure for Tensorboard

In [0]:
!rm -rf /content/logs
!mkdir /content/logs/
!mkdir /content/logs/scalars

# Create Model

Model Parameters

In [0]:
USE_BATCHNORM = True

Convolution with Batch Normalization \\
(Based on https://medium.com/@mannasiladittya/building-inception-resnet-v2-in-keras-from-scratch-a3546c4d93f0)

In [0]:
def conv(x,numfilt,filtsz,batchnorm,strides=1,padding='same',act=True,name=None):
  x = Conv2D(numfilt,filtsz,strides=strides,padding=padding,data_format='channels_last',use_bias=False,name=name+'_conv2d')(x)
  if batchnorm:
    x = BatchNormalization(axis=3,scale=False,name=name+'_conv2d'+'_bn')(x)
  if act:
    x = Activation('relu',name=name+'_conv2d'+'_act')(x)
  return x

Inception Module

In [0]:
def inception_module(x,
                     filters_1x1,
                     filters_3x3_reduce,
                     filters_3x3,
                     filters_5x5_reduce,
                     filters_5x5,
                     filters_pool_proj,
                     name=None):
    
    conv_1x1 = conv(x,filters_1x1, (1, 1), batchnorm=USE_BATCHNORM, name=name+'_1x1')
    
    conv_3x3 = conv(x,filters_3x3_reduce, (1, 1), batchnorm=USE_BATCHNORM, name=name+'_3x3_reduce')
    conv_3x3 = conv(conv_3x3,filters_3x3, (3, 3), batchnorm=USE_BATCHNORM, name=name+'_3x3')

    conv_5x5 = conv(x,filters_5x5_reduce, (1, 1), batchnorm=USE_BATCHNORM, name=name+'_5x5_reduce')
    conv_5x5 = conv(conv_5x5,filters_5x5, (5, 5), batchnorm=USE_BATCHNORM, name=name+'_5x5')

    pool_proj = MaxPool2D((3, 3), strides=(1, 1), padding='same')(x)
    pool_proj = conv(pool_proj,filters_pool_proj, (1, 1), batchnorm=USE_BATCHNORM,name=name+'_pool_proj')

    output = concatenate([conv_1x1, conv_3x3, conv_5x5, pool_proj], axis=3, name=name)
    
    return output

Inception Net \\
(Based on https://www.analyticsvidhya.com/blog/2018/10/understanding-inception-network-from-scratch/)

In [0]:
#Input
x = input_img

# Inception modules
x = inception_module(x,
                     filters_1x1=64,
                     filters_3x3_reduce=96,
                     filters_3x3=128,
                     filters_5x5_reduce=16,
                     filters_5x5=32,
                     filters_pool_proj=32,
                     name='inception_3a')

x = inception_module(x,
                     filters_1x1=128,
                     filters_3x3_reduce=128,
                     filters_3x3=192,
                     filters_5x5_reduce=32,
                     filters_5x5=96,
                     filters_pool_proj=64,
                     name='inception_3b')

x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_3_3x3/2')(x)

x = inception_module(x,
                     filters_1x1=192,
                     filters_3x3_reduce=96,
                     filters_3x3=208,
                     filters_5x5_reduce=16,
                     filters_5x5=48,
                     filters_pool_proj=64,
                     name='inception_4a')

x = inception_module(x,
                     filters_1x1=160,
                     filters_3x3_reduce=112,
                     filters_3x3=224,
                     filters_5x5_reduce=24,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_4b')

x = inception_module(x,
                     filters_1x1=128,
                     filters_3x3_reduce=128,
                     filters_3x3=256,
                     filters_5x5_reduce=24,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_4c')

x = inception_module(x,
                     filters_1x1=112,
                     filters_3x3_reduce=144,
                     filters_3x3=288,
                     filters_5x5_reduce=32,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_4d')

x = inception_module(x,
                     filters_1x1=256,
                     filters_3x3_reduce=160,
                     filters_3x3=320,
                     filters_5x5_reduce=32,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='inception_4e')

x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_4_3x3/2')(x)

x = inception_module(x,
                     filters_1x1=256,
                     filters_3x3_reduce=160,
                     filters_3x3=320,
                     filters_5x5_reduce=32,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='inception_5a')

x = inception_module(x,
                     filters_1x1=384,
                     filters_3x3_reduce=192,
                     filters_3x3=384,
                     filters_5x5_reduce=48,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='inception_5b')

x = GlobalAveragePooling2D(name='avg_pool_5_3x3/1')(x)

# Fully connected Layer
x = Dense(10, activation='softmax', name='output')(x)

In [0]:
model = Model(input_img, x, name='inception_v3')

Output Model

In [0]:
model.summary()
# Comment next line in to plot full model
# plot_model(model, show_shapes=True, to_file='inception_module.png')

# Training

Parameters

In [0]:
BATCH_SIZE = 250
EPOCHS = 30
STEPS=len(X_train)/BATCH_SIZE

INITIAL_LRATE = 0.01
DROP = 0.1
EPOCHS_DROP = 10

New Tensorboard Callback

In [0]:
logdir = "/content/logs/scalars/BN_LR001_BS10_E30_" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=logdir, batch_size=BATCH_SIZE)

Data Augmentation and Train Generator

In [0]:
# Flip, shear and zoom images
train_datagen = ImageDataGenerator(
  horizontal_flip=True,
  shear_range = 0.2,
  zoom_range = 0.2)
train_datagen.fit(X_train)
train_generator = train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE)

Train Model \\
(Based on https://www.analyticsvidhya.com/blog/2018/10/understanding-inception-network-from-scratch/)

In [0]:
# Learning rate decay
def decay(epoch, steps=STEPS):
    initial_lrate = INITIAL_LRATE
    drop = DROP
    epochs_drop = EPOCHS_DROP
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

lr_sc = LearningRateScheduler(decay, verbose=1)

# SGD Optimizer
sgd = SGD(lr=INITIAL_LRATE, momentum=0.9, nesterov=False)

# Compile Model
model.compile(loss= 'categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [0]:
model.fit_generator(train_generator, steps_per_epoch=STEPS, epochs=EPOCHS, validation_data=(X_test, y_test), callbacks=[tensorboard_callback,lr_sc])

Tensorboard

In [0]:
%tensorboard --logdir /content/logs/scalars